<a href="https://colab.research.google.com/github/amitmakashir/Artificial-Intelligence-in-Adversarial-Games/blob/master/Loss_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
import sklearn
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import random
import matplotlib.cm as cm
import math

In [0]:
# a = tf.ones([3,2,2],dtype=tf.dtypes.float32)
# b = tf.zeros([3,2,2],dtype=tf.dtypes.float32)

grows = 3
gcols = 3
col_id = tf.to_float(tf.reshape(tf.tile(tf.range(grows), [gcols]), (1, grows, gcols, 1)))
row_id = tf.to_float(tf.reshape(tf.tile(tf.range(grows), [gcols]), (1, grows, gcols, 1)))
row_id = tf.transpose(row_id)
coord_id = tf.concat([col_id,row_id],-1)

c = tf.multiply(coord_id,[10,10])

d = tf.reverse(c,[-1])
with tf.Session() as sess:
    print(sess.run(d))

[[[[ 0.  0.]
   [ 0. 10.]
   [ 0. 20.]]

  [[10.  0.]
   [10. 10.]
   [10. 20.]]

  [[20.  0.]
   [20. 10.]
   [20. 20.]]]]


In [0]:
# https://medium.com/@jonathan_hui/real-time-object-detection-with-yolo-yolov2-28b1b93e2088
# https://github.com/thtrieu/darkflow/blob/master/darkflow/net/yolo/train.py
# https://hackernoon.com/understanding-yolo-f5a74bbc7967

def loss(y_true,y_pred):
    '''
    y_true.shape => (batch,grows,gcols,params)    \\ params = P(obj) + 4 b.b params + each class probabilities
    y_pred.shape => y_true.shape
        
    b.b params => (x,y,h,w)
    
    There are 3 types of losses:
    1. classification loss
    2. localization loss
    3. confidence loss
    
    The λ parameters that appear here and also in the first part are used to 
    differently weight parts of the loss functions. This is necessary to increase 
    model stability. The highest penalty is for coordinate predictions (λ coord = 5) 
    and the lowest for confidence predictions when no object is present (λ noobj = 0.5).
    '''
    
    ##### Confidence loss
    '''
    Most boxes do not contain any objects. This causes a class imbalance problem,
    i.e. we train the model to detect background more frequently than detecting 
    objects. To remedy this, we weight this loss down by a factor λnoobj (default: 0.5).
    box_conf_score = P(obj) * (IoU with g.t)
    '''
    P_obj = y_pred[:,:,:,0]    # Slice probs
    
    # Calculate iou for all the predicted boxes
    # calculate intersection
    denorm_coords = Denormalize_coordinates(y_pred[:,:,:,1:3]) # Slice the x,y
    true_area,pred_area,interset = Areas(denorm_coords,y_pred[:,:,:,3:5])
    
    lefttop,rightbot = Corner_coords(denorm_coords,y_pred[:,:,:,3:5]) # Slice the h,w
    
    
    iou = tf.truediv(intersect, true_area + pred_area - intersect)  # intersection/union
    
    confs = tf.math.multiply(P_obj,iou)
    
    

In [0]:
def Denormalize_coordinates(pred_coords):
    '''
    Inputs
    pred_coords.shape => (batch,grows,gcols,2) # normed x and y of the b.b
    
    Output:
    denormalized_coord.shape => (batch,grows,gcols,2) # denormed x and y of the b.b
    '''
    img_height = tf.constant(342)
    img_width = tf.constant(342)
    
    grows = tf.constant(19)
    gcols = tf.constant(19)
    
    cell_height = tf.truediv(img_height,grows)
    cell_width = tf.truediv(img_width,gcols)
    
    grid_cell_topleft["y"] = grid_row_no*cell_height
    grid_cell_topleft["x"] = grid_col_no*cell_width
        
    col_id = tf.to_float(tf.reshape(tf.tile(tf.range(grows), [gcols]), (1, grows, gcols, 1)))
    row_id = tf.to_float(tf.reshape(tf.tile(tf.range(grows), [gcols]), (1, grows, gcols, 1)))
    row_id = tf.transpose(row_id)
    coord_id = tf.concat([col_id,row_id],-1) # a 4D tensor that gives the coordinates for every box (col,row)

    coords_lefttop = tf.multiply(coord_id,[cell_width,cell_height]) # Gets the leftmost coordinate for every box
    cell_denormalized_coords = tf.multiply(pred_coords,[cell_width,cell_height])
    
    denormalized_coord = coords_lefttop + cell_denormalized_coords
    
    return denormalized_coord

In [0]:
def Corner_coords(denorm_coords,box_params):
    '''
    We have h,w but we want w,h
    '''
    img_height = tf.constant(342)
    img_width = tf.constant(342)
    
    grows = tf.constant(19)
    gcols = tf.constant(19)
    
    cell_height = tf.truediv(img_height,grows)
    cell_width = tf.truediv(img_width,gcols)
    
    box_params = tf.reverse(box_params,[-1]) # Now w,h
    
    halfs = box_params/2
    
    lefttop = denorm_coords - halfs
    rightbot = denorm_coords + halfs
    
    return lefttop,rightbot

In [0]:
def Areas(denorm_coords,y_true,box_params):
    lefttop,rightbot = Corner_coords(denorm_coords,box_params) # Slice the h,w
    
    
    
    